In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# # Load the Keras model
# keras_model_path = '/path/to/your_model.h5'
# keras_model = load_model(keras_model_path)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit model input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

# Load the X-ray testing dataset
test_dataset_path = './Data/test'  # Replace with your test dataset path
test_dataset = ImageFolder(root=test_dataset_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [4]:
def get_keras_predictions(model, data_loader):
    predictions = []
    with torch.no_grad():
        for images, _ in data_loader:
            images = images.numpy()  # Convert to numpy array
            images = np.transpose(images, (0, 2, 3, 1))  # Change from (N, C, H, W) to (N, H, W, C)
            outputs = model.predict(images)
            predictions.append(outputs)
    return np.concatenate(predictions)


In [8]:
alexnet_model = models.alexnet(pretrained=False)
resnext_model = models.resnext50_32x4d(pretrained=False)
densenet_model = models.densenet121(pretrained=False)

# Load the model weights
alexnet_model.load_state_dict(torch.load('./models/alexnet_model.pth'))
resnext_model.load_state_dict(torch.load('./models/resnext_model.pth'))
densenet_model.load_state_dict(torch.load('./models/densenet_model.pth'))

# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet_model.to(device)
resnext_model.to(device)
densenet_model.to(device)

# Set models to evaluation mode
alexnet_model.eval()
resnext_model.eval()
densenet_model.eval()

c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


AttributeError: Can't get attribute 'alexnet' on <module '__main__'>

In [ ]:

def get_torch_predictions(model, data_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, _ in data_loader:
            images = images.to(device)
            outputs = model(images)
            probabilities = F.softmax(outputs, dim=1)
            predictions.append(probabilities.cpu().numpy())
    return np.concatenate(predictions)


In [ ]:
# Get predictions from each individual model
# keras_predictions = get_keras_predictions(keras_model, test_loader)
alexnet_predictions = get_torch_predictions(alexnet_model, test_loader)
resnext_predictions = get_torch_predictions(resnext_model, test_loader)
densenet_predictions = get_torch_predictions(densenet_model, test_loader)

# Combine predictions using averaging
ensemble_predictions = (alexnet_predictions + resnext_predictions + densenet_predictions) / 4

# Optionally, convert ensemble predictions to labels using argmax
ensemble_labels = np.argmax(ensemble_predictions, axis=1)


In [ ]:
# Load true labels from the test dataset
true_labels = []
for _, labels in test_loader:
    true_labels.extend(labels.numpy())

# Calculate accuracy
accuracy = accuracy_score(true_labels, ensemble_labels)
print("Ensemble Model Accuracy:", accuracy)
